In [1]:
import os #Gestion des fichiers
import nltk #Gestion texte
from nltk.corpus import stopwords #Stopwords
import numpy #Opérations mathématiques
import matplotlib #Graphiques
import string #Gestion de chaine de caractère
from nltk.tokenize import word_tokenize
#from pandas import DataFrame
import pandas #Importer données en DataFrame  
from sklearn.model_selection import train_test_split #Selection abstract pour corpus d’apprentissage  
from sklearn.feature_extraction.text import CountVectorizer #Construction matrice pour apprentissage
from sklearn.linear_model import LogisticRegression #Régression logistique
from sklearn import metrics #Matrice de confusion
import pandas as pd

# Lancer le programme avec l'ensemble des fonctions
%run "P:\SIEGE\FONDATION\DOCUMENTS COMMUNS\Transferts\Analyse de textes\Programmes\Fonctions.ipynb"


"""Import de nos 3 fichiers, dans l'ordre"""

os.chdir(r"P:\SIEGE\FONDATION\DOCUMENTS COMMUNS\Transferts\Analyse de textes")
abstract_embase=import_zot('Zot_Embase_1.txt')
abstract_coch=import_coch('Cochrane_2.txt')
abstract_pub=import_zot('Zot_Pubmed_3.txt')

#Création d'une liste d'abstract total
abstract_tot=abstract_embase+abstract_coch+abstract_pub

#A voir après si on  a le temps de faire cette correction
#abstract_tot.replace("cost effectiveness", "cost-effectiveness")

AttributeError: 'list' object has no attribute 'replace'

In [2]:
#On applique la fonction à notre liste d'abstract, et on split en mots
doc_clean = [clean(doc).split() for doc in abstract_tot]      


In [5]:
#Import du dictionnaire UK/US
dic_uk_us = pd.read_excel (r'P:\SIEGE\FONDATION\DOCUMENTS COMMUNS\Transferts\Analyse de textes\dic_uk_us.xlsx')
# On transforme la première colonne en liste
liste_uk=dic_uk_us['UK'].values.tolist()
liste_us=dic_uk_us['US'].values.tolist()

#On ne garde que l'orthographique britannique pour homogénéiser
doc_clean_bis=[[liste_uk[liste_us.index(mot)] if mot in liste_us else mot for mot in a ] for a in doc_clean]

#Uniformisation de l'orthographe de cost-effectiveness - A VOIR SI ON A LE TEMPS
#doc_clean_tiret=[[mot.replace("costeffectiveness","cost effectiveness") for mot in a] for a in doc_clean_bis]
#doc_clean_tiret2=[[mot.replace("cost-effectiveness","cost effectiveness") for mot in a] for a in doc_clean_tiret]

# On nettoie de nouveau des mots qui semblent inopportuns pour la suite des analyses
motsARetirer = ['background','purpose','p','n','per','also','mm','ci','thus','however','http','ii','iii','et','al','se','cc','email','eo','01','02','03','04','05','06','07','08','09']
doc_clean_nettoye = []

for sous_tableau in doc_clean_bis:
    tableau_nettoye = [mot for mot in sous_tableau if not mot in motsARetirer] 
    doc_clean_nettoye.append(tableau_nettoye)

# rassembler tous les mots d'un même abstract
doc_clean2=[]
for element in doc_clean_nettoye: #changer en doc_clean_nettoye_2 : si chiffres supprimés
    mot_1=" ".join(element)
    doc_clean2.append(mot_1)
#print(mots_nettoyes[0:1])    

#Création d'une liste avec les identifiants allant de 1 à 3211
ident=[str(a) for a in range(1,3212,1)]

#On crée une liste avec des couples clé/abstract que l'on transforme ensuite en dictionnaire    
t=[]
for i in range(0,3211):
    t.append([ident[i],doc_clean2[i]])
    
# Création du dictionnaire
dict_abs=dict(t)
type(dict_abs)

# Importer le fichier xlsx avec les identifiants à conserver 

id_excel = pd.read_excel (r'P:\SIEGE\FONDATION\DOCUMENTS COMMUNS\Transferts\Analyse de textes\Identifiants_a_importer.xlsx', sheet_name='Second écrémage')
#print (id_excel.iloc[:9])

# On trie en fonction de l'identifiant 1
id_excel=id_excel.sort_values('Identifiant_1')

# On transforme la première colonne en liste
liste_ident=id_excel['Identifiant_1'].values.tolist()

# On transforme en str
liste_ident=[str(i) for i in liste_ident] 
#print(liste_ident)


# Eliminer les doublons
article_conserve = {k: dict_abs[k] for k in liste_ident if k in dict_abs}

# Transformation du dictionnaire en dataframe
base_articles=pd.DataFrame(list(article_conserve.items()), columns=['Identifiant_1', 'Abstract'])

# On change le format de la colonne binaire 0/1 et on la fusionne à la base des articles (sans doublons) 
id_excel.Identifiant_1=[str(i) for i in id_excel.Identifiant_1] 
articles_vf=pd.merge(base_articles, id_excel, on=['Identifiant_1']) 

# Supprimer les articles qui ont des abstract manquants
articles_vf=articles_vf[articles_vf.Abstract != 'abstract absent']
len(articles_vf) 


2488